In [ ]:
import os
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
from ultralytics import YOLO
from pdf2image import convert_from_path
import subprocess
from pathlib import Path
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText
import pytesseract

# --- 환경 설정 ---
# GPU 사용 설정 (평가 서버에 GPU가 있으므로 'cuda'로 설정됩니다)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 모델 로딩 (전부 로컬 경로에서 로드) ---

# 1. YOLO 레이아웃 모델 로딩 (제출 폴더 내 'model' 폴더)
layout_model_path = "./model/yolov12l-doclaynet.pt"
layout_model = YOLO(layout_model_path)
print(f"Layout model loaded from: {layout_model_path}")

# 2. Qwen2-VL OCR 모델 로딩 (제출 폴더 내 'models' 폴더)
# 이 경로는 submit.zip 파일 내부 구조와 일치해야 합니다.
local_vlm_path = "./model/qwen2.5-vl-3b-instruct"
print(f"Loading VLM-OCR model from local files: {local_vlm_path}...")

try:
    # 먼저 processor만 로딩 시도
    print("Loading processor...")
    ocr_processor = AutoProcessor.from_pretrained(local_vlm_path, trust_remote_code=True)
    print("Processor loaded successfully.")
    
    # 모델 로딩 시도 - AutoModelForImageTextToText 사용
    print("Loading model...")
    
    # Qwen2-VL-2B-Instruct 모델 로딩
    ocr_model = AutoModelForImageTextToText.from_pretrained(
        local_vlm_path, 
        trust_remote_code=True,
        torch_dtype=torch.float16 if device.type == 'cuda' else torch.float32,
        low_cpu_mem_usage=True
    )
    print("Model loaded successfully")
    
    # GPU로 이동
    if device.type == 'cuda':
        ocr_model = ocr_model.to(device)
        
    print("VLM-OCR model loaded successfully.")
    
except Exception as e:
    print(f"Error loading VLM-OCR model: {e}")
    print("Falling back to basic OCR...")
    ocr_processor = None
    ocr_model = None

# --- 전역 변수 및 설정 ---
# 클래스 이름 매핑
LABEL_MAP = {
    'Text': 'text',
    'Title': 'title',
    'Section-header': 'subtitle',
    'Formula': 'equation',
    'Table': 'table',
    'Picture': 'image'
}

# --- 함수 정의 ---

def convert_to_images(input_path, temp_dir, dpi=200):
    ext = Path(input_path).suffix.lower()
    os.makedirs(temp_dir, exist_ok=True)

    if ext == ".pdf":
        return convert_from_path(input_path, dpi=dpi, output_folder=temp_dir, fmt="png")
    elif ext == ".pptx":
        # Convert pptx to pdf first
        subprocess.run([
            "libreoffice", "--headless", "--convert-to", "pdf", "--outdir", temp_dir, input_path
        ], check=True)
        pdf_path = os.path.join(temp_dir, Path(input_path).with_suffix(".pdf").name)
        return convert_from_path(pdf_path, dpi=dpi, output_folder=temp_dir, fmt="png")
    elif ext in [".jpg", ".jpeg", ".png"]:
        return [Image.open(input_path).convert("RGB")]
    else:
        raise ValueError(f"지원하지 않는 파일 형식입니다: {ext}")

def scale_bbox(bbox, current_size, target_size):
    """Bounding box 좌표를 현재 크기에서 목표 크기로 스케일링합니다."""
    x1, y1, x2, y2 = bbox
    current_w, current_h = current_size
    target_w, target_h = target_size
    
    if current_w == 0 or current_h == 0: return [0, 0, 0, 0]
        
    scale_x = target_w / current_w
    scale_y = target_h / current_h
    return [
        int(x1 * scale_x), int(y1 * scale_y),
        int(x2 * scale_x), int(y2 * scale_y)
    ]

def extract_text_with_vlm(image_pil, bbox, debug_info=None):
    """이미지의 특정 bbox 영역을 잘라 VLM으로 OCR을 수행합니다."""
    x1, y1, x2, y2 = bbox
    img_w, img_h = image_pil.size
    # bbox 좌표가 이미지 경계를 벗어나지 않도록 보정
    x1, y1, x2, y2 = max(0, x1), max(0, y1), min(img_w, x2), min(img_h, y2)
    
    if x1 >= x2 or y1 >= y2: return ""
        
    cropped_image = image_pil.crop((x1, y1, x2, y2))

    # 너무 작은 이미지는 처리하지 않음
    if cropped_image.width < 10 or cropped_image.height < 10: return ""
    
    # 크롭된 이미지 시각화 저장 (디버깅용)
    if debug_info is not None:
        debug_dir = "./debug_crops"
        os.makedirs(debug_dir, exist_ok=True)
        crop_filename = f"{debug_info['id']}_{debug_info['category']}_{debug_info['order']}.png"
        crop_path = os.path.join(debug_dir, crop_filename)
        cropped_image.save(crop_path)
        print(f"🔍 Cropped image saved: {crop_path} (size: {cropped_image.size})")

    # VLM이 로드되지 않았으면 pytesseract 사용
    if ocr_processor is None or ocr_model is None:
        try:
            # pytesseract로 OCR 수행
            text = pytesseract.image_to_string(cropped_image, lang='kor+eng').strip()
            print(f"🔤 Pytesseract OCR result: '{text}' (from {debug_info['id']}_{debug_info['category']}_{debug_info['order']})")
            return text
        except Exception as e:
            print(f"Error during pytesseract OCR: {e}")
            return ""

    # 더 강력한 OCR 프롬프트로 수정
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": cropped_image,
                },
                {"type": "text", "text": "Extract all text visible in this image. Return only the raw text without any explanations or apologies. If no text is visible, return empty."},
            ],
        }
    ]
    
    try:
        # Apply chat template
        text = ocr_processor.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        
        # Process inputs
        inputs = ocr_processor(
            text=[text],
            images=[cropped_image],
            return_tensors="pt",
            padding=True,
        )
        
        # GPU 사용 시에만 cuda로 이동
        if device.type == 'cuda':
            inputs = inputs.to(device)
        
        # Generate response
        with torch.no_grad():
            generated_ids = ocr_model.generate(
                **inputs,
                max_new_tokens=256,
                do_sample=False,
                temperature=0.1,
                pad_token_id=ocr_processor.tokenizer.eos_token_id,
                repetition_penalty=1.1
            )
        
        # Decode response
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)
        ]
        response = ocr_processor.batch_decode(
            generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
        )[0]
        return response.strip()
        
    except Exception as e:
        print(f"Error during VLM OCR on a cropped image: {e}")
        # fallback to pytesseract
        try:
            text = pytesseract.image_to_string(cropped_image, lang='kor+eng').strip()
            print(f"🔤 (fallback) Pytesseract OCR: '{text}'")
            return text
        except Exception as e2:
            print(f"Error during pytesseract fallback: {e2}")
            return ""

def visualize_predictions(image_pil, predictions, save_path):
    """예측 결과를 이미지에 시각화해서 저장합니다."""
    # 이미지 복사본 생성
    vis_image = image_pil.copy()
    draw = ImageDraw.Draw(vis_image)
    
    # 카테고리별 색상 정의
    colors = {
        'title': 'red',
        'subtitle': 'orange', 
        'text': 'blue',
        'equation': 'green',
        'table': 'purple',
        'image': 'pink'
    }
    
    # 폰트 설정 (기본 폰트 사용)
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except:
        font = ImageFont.load_default()
    
    for pred in predictions:
        bbox_str = pred['bbox']
        x1, y1, x2, y2 = map(int, bbox_str.split(','))
        category = pred['category_type']
        confidence = pred['confidence_score']
        order = pred['order']
        
        # 바운딩 박스 그리기
        color = colors.get(category, 'gray')
        draw.rectangle([x1, y1, x2, y2], outline=color, width=3)
        
        # 라벨 텍스트
        label = f"{category}_{order} ({confidence:.2f})"
        
        # 텍스트 배경 박스
        text_bbox = draw.textbbox((x1, y1-25), label, font=font)
        draw.rectangle(text_bbox, fill=color)
        draw.text((x1, y1-25), label, fill='white', font=font)
    
    # 저장
    vis_image.save(save_path)
    print(f"📊 Visualization saved: {save_path}")

def inference_one_image(id_val, image_pil, target_size, conf_thres=0.15, imgsz=1920):
    """단일 이미지에 대해 레이아웃 감지 및 OCR 추론을 수행합니다."""
    original_size = image_pil.size
    
    # YOLO 추론을 위해 이미지 리사이즈 (YOLO 모델 학습 시 사용된 크기)
    # 임시 파일 저장을 통해 메모리 문제를 완화할 수 있음
    temp_path = "_temp_image.png"
    image_pil.resize((imgsz, imgsz)).save(temp_path)

    results = layout_model(
        source=temp_path, 
        imgsz=imgsz, 
        conf=conf_thres, 
        iou=0.3,  # NMS IoU threshold - 낮을수록 더 많은 박스 유지
        agnostic_nms=True,  # class-agnostic NMS
        max_det=300,  # 최대 감지 수 증가
        verbose=False
    )[0]
    os.remove(temp_path)

    predictions = []
    if results.boxes is None: return []

    # Bbox를 y좌표 기준으로 정렬하여 문서의 위->아래 순서로 처리
    sorted_boxes = sorted(zip(results.boxes.xyxy, results.boxes.conf, results.boxes.cls), key=lambda x: x[0][1])

    for order, (box, score, cls) in enumerate(sorted_boxes):
        label = results.names[int(cls)]
        if label not in LABEL_MAP: continue
            
        category_type = LABEL_MAP[label]
        
        # 1. 추론된 bbox(imgsz x imgsz 기준)를 원본 이미지 크기로 변환
        original_bbox = scale_bbox(box.tolist(), (imgsz, imgsz), original_size)
        
        text = ''
        if category_type in ['title', 'subtitle', 'text']:
            debug_info = {
                'id': id_val,
                'category': category_type, 
                'order': order
            }
            text = extract_text_with_vlm(image_pil, original_bbox, debug_info)
        
        # 2. 원본 기준 bbox를 최종 제출 형식(target_size)에 맞게 변환
        final_bbox = scale_bbox(original_bbox, original_size, target_size)

        predictions.append({
            'ID': id_val,
            'category_type': category_type,
            'confidence_score': score.cpu().item(),
            'order': order,
            'text': text,
            'bbox': f'{final_bbox[0]},{final_bbox[1]},{final_bbox[2]},{final_bbox[3]}'
        })
    
    # 시각화 저장 (디버깅용)
    debug_vis_dir = "./debug_visualizations"
    os.makedirs(debug_vis_dir, exist_ok=True)
    vis_filename = f"{id_val}_visualization.png"
    vis_path = os.path.join(debug_vis_dir, vis_filename)
    
    # 원본 이미지 크기로 bbox 변환해서 시각화
    vis_predictions = []
    for pred in predictions:
        vis_pred = pred.copy()
        # target_size 기준 bbox를 원본 크기로 다시 변환
        bbox_coords = list(map(int, pred['bbox'].split(',')))
        original_bbox_coords = scale_bbox(bbox_coords, target_size, original_size)
        vis_pred['bbox'] = f'{original_bbox_coords[0]},{original_bbox_coords[1]},{original_bbox_coords[2]},{original_bbox_coords[3]}'
        vis_predictions.append(vis_pred)
    
    visualize_predictions(image_pil, vis_predictions, vis_path)
    
    return predictions

def main_inference(test_csv_path, output_csv_path, conf_thres=0.15, imgsz=1920):
    """메인 추론 함수: CSV를 읽고, 모든 파일에 대해 추론을 실행하며, 결과를 저장합니다."""
    output_dir = os.path.dirname(output_csv_path)
    os.makedirs(output_dir, exist_ok=True)
    temp_image_dir = "./temp_images"
    os.makedirs(temp_image_dir, exist_ok=True)
    
    csv_dir = os.path.dirname(test_csv_path)
    test_df = pd.read_csv(test_csv_path)
    all_preds = []

    print(f"Using confidence threshold: {conf_thres}, image size: {imgsz}")

    for _, row in test_df.iterrows():
        id_val = row['ID']
        raw_path = row['path']
        file_path = os.path.normpath(os.path.join(csv_dir, raw_path))
        target_size = (int(row['width']), int(row['height']))

        if not os.path.exists(file_path):
            print(f"⚠️ File not found: {file_path}")
            continue

        try:
            images = convert_to_images(file_path, temp_image_dir)
            for i, image in enumerate(images):
                # 멀티페이지 문서 ID 형식 (예: doc1_p1, doc1_p2)
                full_id = f"{id_val}_p{i+1}" if len(images) > 1 else id_val
                preds = inference_one_image(full_id, image, target_size, conf_thres=conf_thres, imgsz=imgsz)
                all_preds.extend(preds)
            print(f"✅ Inference complete: {file_path}")
        except Exception as e:
            print(f"❌ Processing failed: {file_path} -> {e}")

    result_df = pd.DataFrame(all_preds)
    result_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
    print(f"✅ Submission file saved: {output_csv_path}")


# --- 스크립트 실행 ---
if __name__ == "__main__":
    # 대회 규정에 맞는 경로 설정
    # test.csv는 보통 /data/test.csv 에 위치합니다.
    # 제출 파일은 /output/submission.csv 에 저장해야 합니다.
    data_dir = os.environ.get('DATA_DIR', './data')
    output_dir = os.environ.get('OUTPUT_DIR', './output')
    
    # 환경 변수가 없는 로컬 테스트를 위해 폴더 생성
    os.makedirs(output_dir, exist_ok=True)

    test_csv_file = os.path.join(data_dir, 'test.csv')
    submission_file = os.path.join(output_dir, 'submission.csv')

    # 성능 튜닝을 위한 파라미터
    # conf_thres: 0.15 (낮음) ~ 0.5 (높음) - 낮을수록 더 많은 박스 검출
    # imgsz: 1280, 1536, 1920 - 클수록 정확도 향상, 속도 감소
    conf_threshold = 0.15  # 더 낮춰서 더 많은 박스 검출
    image_size = 1920     # 더 크게 해서 정확도 향상
    
    print(f"Performance settings: conf_thres={conf_threshold}, imgsz={image_size}")
    main_inference(
        test_csv_path=test_csv_file, 
        output_csv_path=submission_file,
        conf_thres=conf_threshold,
        imgsz=image_size
    )